#  The LightCurve class

##  Background
#### What kind of physical data are we representing and what do these quantities mean?

Astrophysical variable sources including transients like supernovae are sources that are (roughly) fixed in position but change appreciably at human time scales, such that their measurements at different points of time in a survey are expected to be different. The key science of such sources has to do with interpreting such data.

In a photometric or imaging survey, the raw data associated with such measurements is in the form of a time series of images, along with metadata. This is often extracted into a catalog summarizing the inferred brightness of the source in the filter band and the uncertainties of the inferred. The uncertainties in this process are dominated by the sky noise which is Poisson distributed to a good approximation. Hence, the format of this recorded data is usually of the form

`timestamp`, `flux`, `flux uncertainty`, `zero point`, `magnitude system`

Note that the `flux` is really the band flux obtained in a filter band

The `zero point`, and `magnitude system` are methods of specifying the units of the reported `flux`, and `flux uncertainty`. `flux` may be reported so that the following equation with `bandflux` and `std_bandflux` in the same physical units holds. The `std_bandflux` depends on the transmissions and the magnitude system used.

bandmag := -2.5 log10(bandflux / std_bandflux) = -2.5 log10(flux) + zero point

so that a zero point of zero, implies that `flux` is `bandflux` in units of `std_bandflux`

## Desired Functionality

the `LightCurve` class is meant to be a standard internal representation of this kind of catalog data in our software. This means that 
- given a different format/model of data, we will first convert it into an instance of this class.
- All our calculations will be defined on instances of this class
- We should have output methods to convert the instances of this class to other useful output formats

## Structure

In [1]:
import sncosmo
import analyzeSN as ans

This requires a THROUGHPUTS directory to be setup as an env var

This may be achieved, for example, by setting up the LSST stack

or by cloning the throughputs directory from git-lfs, 

and setting up env vars. The function is to provide the LSST bandpass

files



In [2]:
from analyzeSN import LightCurve

#### Instantiation

The class is instantiated using a constructor that has a mandatory input of a dataframe. and optional inputs. The dataframe must contain the light curve information, and the minimum set of columns required are:
`mjd`, `band`, `flux`, `flux_err`, `zp`, `zpsys` or recognized aliases. Here, we load some data and demonstrate this

In [13]:
ex_data = sncosmo.load_example_data().to_pandas()
ex_data.head()

,time,band,flux,fluxerr,zp,zpsys
0,55070.000000,sdssg,0.363512,0.672844,25.0,ab
1,55072.051282,sdssr,-0.200801,0.672844,25.0,ab
2,55074.102564,sdssi,0.307494,0.672844,25.0,ab
3,55076.153846,sdssz,1.087761,0.672844,25.0,ab
4,55078.205128,sdssg,-0.436679,0.672844,25.0,ab


This was a SNCosmo example dataset, loaded into `pandas`. Note that the column representing temporal information is `time`. Now, we will use this to instantiate our class.

In [4]:
lc = LightCurve(ex_data)

Now, we can see the light curve by using the attribute `lightCurve` which outputs a dataFrame in our convention. We note that `time` has changed to our standard name `mjd`

In [5]:
lc.lightCurve.head()

,mjd,band,flux,fluxerr,zp,zpsys
0,55070.000000,sdssg,0.363512,0.672844,25.0,ab
1,55072.051282,sdssr,-0.200801,0.672844,25.0,ab
2,55074.102564,sdssi,0.307494,0.672844,25.0,ab
3,55076.153846,sdssz,1.087761,0.672844,25.0,ab
4,55078.205128,sdssg,-0.436679,0.672844,25.0,ab


What would have happened if an essential column was missing? to check let us do the following:
   - We create a dataframe like the above, but remove the essential column band
   - We attempt to create an instance of the LightCurve class

In [14]:
lc_tmp = ex_data.copy()
del lc_tmp['band']
lc_tmp.head()

,time,flux,fluxerr,zp,zpsys
0,55070.000000,0.363512,0.672844,25.0,ab
1,55072.051282,-0.200801,0.672844,25.0,ab
2,55074.102564,0.307494,0.672844,25.0,ab
3,55076.153846,1.087761,0.672844,25.0,ab
4,55078.205128,-0.436679,0.672844,25.0,ab


In [15]:
lc_bad = LightCurve(lc_tmp)

ValueError: ('light curve data has missing columns', set(['band']))

This raises a ValueError pointing out that the column `band` was missing. However, it correctly recognizes that while
`mjd` was missing, it had the aliased column `time` instead and did not report it missing.

To get back the SNCosmo light curve data

In [15]:
SNCosmoLC = lc.snCosmoLC()

In [16]:
SNCosmoLC

time,band,flux,fluxerr,zp,zpsys
float64,str5,float64,float64,float64,str2
55070.0,sdssg,0.36351153597,0.672843847541,25.0,ab
55072.0512821,sdssr,-0.200801295864,0.672843847541,25.0,ab
55074.1025641,sdssi,0.307494232981,0.672843847541,25.0,ab
55076.1538462,sdssz,1.08776103656,0.672843847541,25.0,ab
55078.2051282,sdssg,-0.43667895645,0.672843847541,25.0,ab
55080.2564103,sdssr,1.09780966779,0.672843847541,25.0,ab
55082.3076923,sdssi,3.7562685627,0.672843847541,25.0,ab
55084.3589744,sdssz,5.34858894966,0.672843847541,25.0,ab
55086.4102564,sdssg,2.82614187269,0.672843847541,25.0,ab


In [1]:
10**(-0.4 * 0.27)

0.7798301105232587

In [2]:
-2.5 * np.log10(f) 

SyntaxError: can't assign to operator (<ipython-input-2-b3135cee020e>, line 1)

In [3]:
10.0**(0.27 / 2.5 )

1.2823305826560216